# Import

In [1]:
import pandas as pd
import numpy as np
import pygaze
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy
import glob
from tqdm import tqdm
from sklearn.cluster import DBSCAN
import detectors
import gazeplotter
from collections import defaultdict
# import local lib
import eye_metrics_utils
import data_utils
import gaze_entropy

In [2]:
import warnings
# warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')

In [3]:
csv_files = glob.glob("data/*.csv")

In [4]:
csv_files_one = [v for v in csv_files if "One Gaze-Vergence" in v]
csv_files_two = [v for v in csv_files if "Two Gaze-Vergence" in v]
csv_files_three = [v for v in csv_files if "Three Go-Around Gaze-Vergence" in v]

In [5]:
df_par = pd.read_csv("participant.csv")
group = [df_par[df_par['Group'].str.contains("1")]['ID'].tolist(), df_par[df_par['Group'].str.contains("2")]["ID"].tolist()]
group = [[i[-3:] for i in v] for v in group]
group

[['032', '027', '031', '028', '004', '008', '010', '029', '003', '007', '023'],
 ['021',
  '006',
  '019',
  '022',
  '015',
  '016',
  '014',
  '005',
  '025',
  '002',
  '001',
  '020',
  '011',
  '017']]

In [6]:
def run_all(df_data):
    df_x = df_data.copy()
    if (data_utils.check_percentage_null(df_x) < 0.5): # if missing value > 50%, remove
        return None
    
    time = np.array(df_data['Start Time (secs)'].tolist())

    Efix = eye_metrics_utils.detect_fixations(df_x)
#     print(Efix)
    X = np.array(Efix).T[3:].T
    Hs, Ht = gaze_entropy.entropy(X)
    total_time = time[-1] - time[0]
    
    return Efix, Hs, Ht, total_time
    

In [7]:
feature_groups = []
for g in group:
    trials = []
    for csv_files in [csv_files_one, csv_files_two, csv_files_three]:
        ret = defaultdict(list)
        for csv in csv_files:
            par_id = csv[14:17]
            if par_id not in g:
                continue
                
            print(csv)
            df_data = pd.read_csv(csv)
            print(len(df_data))
            for v in data_utils.data_slicing(df_data):
                r = run_all(v)
                if r != None:
                    Efix, Hs, Ht, total_time = r
#                     ret["Eblk"].append(Eblk)
                    ret["Efix"].append(Efix)
#                     ret["Esac"].append(Esac)
#                     ret["trans_matrix"].append(trans_matrix)
                    ret["Hs"].append(Hs)
                    ret["Ht"].append(Ht)
                    ret["total_time"].append(total_time)
        trials.append(ret)
    feature_groups.append(trials)

data\PISSS_ID_003_Approach One Gaze-Vergence.csv
9122
data\PISSS_ID_004_Approach One Gaze-Vergence.csv
9307
data\PISSS_ID_007_Approach One Gaze-Vergence.csv
9492
data\PISSS_ID_008_Approach One Gaze-Vergence.csv
9736
data\PISSS_ID_010_Approach One Gaze-Vergence.csv
9554
data\PISSS_ID_023_Approach One Gaze-Vergence.csv
9369
data\PISSS_ID_027_Approach One Gaze-Vergence.csv
9060
data\PISSS_ID_028_Approach One Gaze-Vergence.csv
8999
data\PISSS_ID_029_Approach One Gaze-Vergence.csv
9862
data\PISSS_ID_031_Approach One Gaze-Vergence.csv
9677
data\PISSS_ID_032_Approach One Gaze-Vergence.csv
8629
data\PISSS_ID_003_Approach Two Gaze-Vergence.csv
9368
data\PISSS_ID_004_Approach Two Gaze-Vergence.csv
9862
data\PISSS_ID_007_Approach Two Gaze-Vergence.csv
9677
data\PISSS_ID_008_Approach Two Gaze-Vergence.csv
9923
data\PISSS_ID_010_Approach Two Gaze-Vergence.csv
9923
data\PISSS_ID_023_Approach Two Gaze-Vergence.csv
9677
data\PISSS_ID_027_Approach Two Gaze-Vergence.csv
8568
data\PISSS_ID_028_Approach T

# function

In [ ]:
def get_center(clustering, data):
    center = []
    for i in range(len(set(clustering.labels_)) - 1):
        xi = data[np.where(clustering.labels_ == i)]
        cx = sum(xi.T[0])/len(xi)
        cy = sum(xi.T[1])/len(xi)
        center.append((cx,cy))
    return center

# cluster_center = get_center(clustering,X)

In [ ]:
def transition_matrix(transitions):
    if len(transitions) == 0:
        return 0
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M


In [ ]:
def distance(x,y):
    return ((x[0]-y[0])**2 + (x[1]-y[1])**2)**0.5

def dbscan_predict(cluster_center, X_new, min_dist = 50, metric=distance):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        for i, x_core in enumerate(cluster_center): 
            if metric(x_new, x_core) < min_dist:
                # Assign label of x_core to x_new
                y_new[j] = i
                break

    return y_new


def dbscan_predict2(cluster_center, X_new, min_dist = 50, metric=distance):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        dist = [metric(x_new, v) for v in cluster_center]
#         print("dist", dist)
#         print("argmin", np.argmin(dist))
        if len(dist) == 0:
            continue
        y_new[j] = np.argmin(dist)

    return y_new

In [ ]:
def data_slicing(df_data, window_length = 1200, stride = 300):
    L = len(df_data)
    for i in range(L//stride):
        df_slice = df_data.iloc[i*stride:i*stride+window_length].copy()
        if len(df_slice) < 600:
            return
        else:
            yield df_slice

In [ ]:
def check_percentage_null(df_data, missing = 0.0):
    return len(df_data[df_data["X Pos"] != 0])/len(df_data)

In [ ]:
def run_all(df_data):
    df_data.fillna(0.0, inplace=True)
    if (check_percentage_null(df_data) < 0.5): # if missing value > 50%, remove
        return None

    x = np.array(df_data['X Pos'].tolist())
    y = np.array(df_data['Y Pos'].tolist())
    time = np.array(df_data['Start Time (secs)'].tolist())*1000
    
    # detect blink, fixation and saccade
    Sblk, Eblk = detectors.blink_detection(x,y,time,minlen=6)
    Sfix, Efix = detectors.fixation_detection(x,y,time,maxdist=5,mindur=50)
    Ssac, Esac = detectors.saccade_detection(x,y,time,minlen=5,maxvel=40,maxacc=340)
    
    # clustering
    X = np.array(Efix).T[3:].T
    clustering = DBSCAN(eps=20, min_samples=3, metric = distance).fit(X)
    cluster_center = get_center(clustering,X)
    pred = dbscan_predict2(cluster_center, np.array(Efix).T[3:].T)
    transitions = pred[np.where(pred!=-1)]
    
    # transition matrix and GTE, SGE
    trans_matrix = transition_matrix(transitions)
    Ht = 0
    Hs = 0
    if trans_matrix != 0:
        pA = [len(np.where(np.array(transitions)==i)[0])/len(transitions) for i in range(len(set(transitions)))]
        for i in range(len(pA)):
            Hs += -1 * np.nan_to_num(pA[i]*np.log2(pA[i]))
            t = np.nan_to_num(trans_matrix[i]*np.log2(trans_matrix[i]))
            Ht += -sum(pA[i]*(t))
    
    total_time = time[-1] - time[0]
        
    return Eblk, Efix, Esac, trans_matrix, Hs, Ht, total_time
    

# import data

In [ ]:
csv_files = glob.glob("data/*.csv")

In [ ]:
csv_files_one = [v for v in csv_files if "One Gaze-Left" in v]
csv_files_two = [v for v in csv_files if "Two Gaze-Left" in v]
csv_files_three = [v for v in csv_files if "Three Go-Around Gaze-Left" in v]

In [ ]:
df_par = pd.read_csv("participant.csv")
group = [df_par[df_par['Group'].str.contains("1")]['ID'].tolist(), df_par[df_par['Group'].str.contains("2")]["ID"].tolist()]
group = [[i[-3:] for i in v] for v in group]
group

In [ ]:
feature_groups = []
for g in group:
    trials = []
    for csv_files in [csv_files_one, csv_files_two, csv_files_three]:
        ret = defaultdict(list)
        for csv in csv_files:
            par_id = csv[14:17]
            if par_id not in g:
                continue
                
            print(csv)
            df_data = pd.read_csv(csv)
            print(len(df_data))
            for v in data_slicing(df_data):
                r = run_all(v)
                if r != None:
                    Eblk, Efix, Esac, trans_matrix, Hs, Ht, total_time = r
                    ret["Eblk"].append(Eblk)
                    ret["Efix"].append(Efix)
                    ret["Esac"].append(Esac)
                    ret["trans_matrix"].append(trans_matrix)
                    ret["Hs"].append(Hs)
                    ret["Ht"].append(Ht)
                    ret["total_time"].append(total_time)
        trials.append(ret)
    feature_groups.append(trials)

In [ ]:
df_data = pd.read_csv("data\PISSS_ID_004_Approach Two Gaze-Left.csv")
df_slice = df_data.iloc[0:3000].copy()

df_slice.fillna(0.0, inplace=True)
print(check_percentage_null(df_slice))

x = np.array(df_slice['X Pos'].tolist())
y = np.array(df_slice['Y Pos'].tolist())
time = np.array(df_slice['Start Time (secs)'].tolist())*1000

# detect blink, fixation and saccade
Sblk, Eblk = detectors.blink_detection(x,y,time,minlen=6)
Sfix, Efix = detectors.fixation_detection(x,y,time,maxdist=10,mindur=50)
Ssac, Esac = detectors.saccade_detection(x,y,time,minlen=5,maxvel=40,maxacc=340)


In [ ]:
feature_groups[1][2]['trans_matrix']

# hypothesis test

In [8]:
# standardized effect size - cohen's d 
def effect_size(a, b):
    es = np.abs(np.mean(a) - np.mean(b))
    sd_pooled = np.sqrt((((len(a)-1)*(np.std(a)**2) + (len(b)-1)*(np.std(b)**2)) / (len(a) + len(b) - 2)))
    d = es/sd_pooled
    
    
    return d

In [9]:
def statistic(g):
    print("GROUP 1")
    print("mean trial 1:", np.mean(g[0][0]))
    print("mean trial 2:", np.mean(g[0][1]))
    print("mean trial 3:", np.mean(g[0][2]))

    print("\nstd trial 1:", np.std(g[0][0]))
    print(  "std trial 2:", np.std(g[0][1]))
    print(  "std trial 3:", np.std(g[0][2]))

    print("--------------------------")
    print("GROUP 2")
    print("mean trial 1:", np.mean(g[1][0]))
    print("mean trial 2:", np.mean(g[1][1]))
    print("mean trial 3:", np.mean(g[1][2]))

    print("\nstd trial 1:", np.std(g[1][0]))
    print(  "std trial 2:", np.std(g[1][1]))
    print(  "std trial 3:", np.std(g[1][2]))

In [66]:
def statistic2(g):
    m = np.array([[np.mean(v) for v in u] for u in g]).reshape(-1)
    std = np.array([[np.std(v) for v in u] for u in g]).reshape(-1)
    group = [1,1,1,2,2,2]
    trial = [1,2,3,1,2,3]
    df = pd.DataFrame(zip(group,trial,m,std), columns=['group',"trial","mean","std"])
    return df

In [154]:
def test(g):
    t = []
    p = []
    e = []
    for i in range(3):
        x = scipy.stats.ttest_ind(g[0][i], g[1][i], equal_var = False)
        t.append(x[0])
        p.append(x[1])
        e.append(effect_size(g[0][i],g[1][i]))
        
    return t,p,e

## fixation count

In [160]:
def fixation_rate():
    g = []
    for v in feature_groups:
        x = np.array([len(v) for v in v[0]['Efix']])/np.array(v[0]['total_time'])
        y = np.array([len(v) for v in v[1]['Efix']])/np.array(v[1]['total_time'])
        z = np.array([len(v) for v in v[2]['Efix']])/np.array(v[2]['total_time'])

        g.append([x,y,z])
    return g
g = fixation_rate()

In [161]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.365731,0.367258
1,1,2,1.352569,0.392210
2,1,3,1.519482,0.253945
3,2,1,1.377188,0.338079
4,2,2,1.361705,0.385604
5,2,3,1.733203,0.356156


In [162]:
scipy.stats.ttest_ind(g[0][2], g[1][2], equal_var = False)

Ttest_indResult(statistic=-3.8833554014938945, pvalue=0.00016744525389689624)

In [163]:
effect_size(g[0][0],g[1][0])

0.032781345831858946

## fixation duration

In [165]:
def fixation_duration():

    g = []
    for v in feature_groups:
        x = np.array([])
        for i, p in enumerate(v[0]['Efix']):
            x = np.append(x,np.array(p).T[2])

        y = []
        for i, p in enumerate(v[1]['Efix']):
            y = np.append(y,np.array(p).T[2])

        z = []
        for i, p in enumerate(v[2]['Efix']):
            z = np.append(z,np.array(p).T[2])

        g.append([x,y,z])
        
    return g 
g= fixation_duration()

In [166]:
statistic2(g)

,group,trial,mean,std
0,1,1,0.459195,0.573703
1,1,2,0.467804,0.589109
2,1,3,0.430915,0.403759
3,2,1,0.450095,0.524439
4,2,2,0.509107,0.631378
5,2,3,0.347214,0.304077


In [167]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=-4.668203642323796, pvalue=3.05970619950258e-06)

In [168]:
effect_size(g[0][1],g[1][1])

0.06734800591613645

## SGE

In [170]:
def sge():

    g = []
    for v in feature_groups:
        x = np.array(v[0]['Hs'])
        x=x[x!=0]

        y = np.array(v[1]['Hs'])
        y=y[y!=0]

        z = np.array(v[2]['Hs'])
        z=z[z!=0]

        g.append([x,y,z])
        
    return g

g = sge()

In [171]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.470392,0.430294
1,1,2,1.436761,0.449737
2,1,3,1.465232,0.420626
3,2,1,1.509409,0.470570
4,2,2,1.375990,0.461782
5,2,3,1.754212,0.417250


In [107]:
scipy.stats.ttest_ind(g[0][1], g[1][1], equal_var = False)

Ttest_indResult(statistic=1.683529535643731, pvalue=0.09280117135004161)

In [108]:
effect_size(g[0][1],g[1][1])

0.13301919936490914

## GTE

In [172]:
def gte():
    g = []
    for v in feature_groups:
        x = np.array(v[0]['Ht'])
        x=x[x!=0]

        y = np.array(v[1]['Ht'])
        y=y[y!=0]

        z = np.array(v[2]['Ht'])
        z=z[z!=0]

        g.append([x,y,z])
        
    return g

g= gte()

In [116]:
statistic2(g)

,group,trial,mean,std
0,1,1,1.189392,0.285081
1,1,2,1.176678,0.327412
2,1,3,1.199061,0.297187
3,2,1,1.206099,0.322813
4,2,2,1.127057,0.336034
5,2,3,1.357890,0.256590


In [113]:
scipy.stats.ttest_ind(g[0][2], g[1][2], equal_var = False)

Ttest_indResult(statistic=-3.0015896333874283, pvalue=0.0034201852814666083)

In [114]:
effect_size(g[0][1],g[1][1])

0.1492332516643208

,trial,group 1 (mean±std),group 2 (mean±std)
0,1,1.37±0.37,1.38±0.34
1,2,1.35±0.39,1.36±0.39
2,3,1.52±0.25,1.73±0.36


In [180]:
# g = fixation_rate()
# g = fixation_duration()
# g = sge()
g = gte()

In [181]:
m = np.array([[np.mean(v) for v in u] for u in g])
std = np.array([[np.std(v) for v in u] for u in g])
# group = [[1,1,1],[2,2,2]]
trial = [1,2,3]
m0 = [(str(round(v[0], 2)) + "±" + str(round(v[1], 2))) for v in zip(m[0],std[0])]
m1 = [(str(round(v[0], 2)) + "±" + str(round(v[1], 2))) for v in zip(m[1],std[1])]

df = pd.DataFrame(zip(trial,m0,m1,), columns=["trial","group 1 (mean±std)","group 2 (mean±std)"])

t, p, e = test(g)

df["t"] = t
df['p'] = p
df['effect size'] = e
df

,trial,group 1 (mean±std),group 2 (mean±std),t,p,effect size
0,1,1.19±0.29,1.21±0.32,-0.678230,0.497919,0.053970
1,2,1.18±0.33,1.13±0.34,1.888595,0.059436,0.149233
2,3,1.2±0.3,1.36±0.26,-3.001590,0.003420,0.578355
